# Sistema basado en reglas y ML
### Simula el comportamiento de los managers de béisbol durante los partidos.

In [ ]:
import pandas as pd

### Simulador de jugadas:

In [1]:
from game_simulator import *

### Reglas

In [2]:
from rules import *

### Manager

In [3]:
from baseball_manager import *

Crear el manager y agregar las reglas


In [4]:
manager = BaseballManager()
manager.add_rule(change_pitcher_rule)
manager.add_rule(steal_base_rule)
manager.add_rule(bunt_rule)
manager.add_rule(challenge_rule)
manager.add_rule(defensive_shift_rule)
manager.add_rule(bullpen_usage_rule)
manager.add_rule(pinch_hitter_rule)
manager.add_rule(hit_and_run_rule)
manager.add_rule(infield_in_rule)
manager.add_rule(defensive_positioning_rule)

# Cargar datos de bateadores y lanzadores
batters_df = pd.read_csv('batters.csv')
pitchers_df = pd.read_csv('pitchers.csv')

# Crear el simulador y simular el juego
simulator = GameSimulator(manager, batters_df, pitchers_df)
simulator.simulate_game(100)  # Simular 100 lanzamientos
simulator.save_log('game_log.json')

# Evaluar las reglas
action = manager.evaluate_rules(simulator.game_state)
print(f"Action: {action}")
# Imprimir el estado del juego

print(simulator)


Árbol de decisión para predecir el rendimiento de los lanzadores.

In [5]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np

# Datos de entrenamiento (ejemplo)
X_train = np.array([[90, 0], [110, 1], [95, 0], [105, 1]])  # [pitch_count, opponent_batter (0: right, 1: left)]
y_train = np.array([0, 1, 0, 1])  # 0: no change, 1: change

# Entrenar el modelo
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Función para predecir la acción
def predict_action(game_state):
    features = np.array([[game_state['pitch_count'], game_state['opponent_batter']]])
    prediction = model.predict(features)
    return "Change pitcher" if prediction[0] == 1 else "No change"


Integrar el Sistema Basado en Reglas y el Modelo de ML

In [6]:
class HybridBaseballManager(BaseballManager):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def evaluate_rules_and_model(self, game_state):
        rule_action = self.evaluate_rules(game_state)
        if rule_action != "No action":
            return rule_action
        return predict_action(game_state)

# Crear el manager híbrido
hybrid_manager = HybridBaseballManager(model)
game_state = simulator.game_state 
action = hybrid_manager.evaluate_rules_and_model(game_state)
print(f"Action: {action}")
